In [65]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, DoubleType, IntegerType
from pyspark.sql.functions import col, struct, explode, when, lit



In [7]:
# load and output directory
# vep_srcdir = 's3://dig-analysis-data/out/varianteffect/effects/part-*'
# outdir = 's3://dig-bio-index/burden/vepbinning'

# test directories
vep_srcdir = '/Users/mduby/Data/Broad/Aggregator/BurdenBinning/test*'
outdir = '/Users/mduby/Data/Broad/Aggregator/BurdenBinning/Out'




In [131]:
# constants for filters
# there are 3 levels of filters (lof, impact + maf, and combined predictions)
# the 7 bins will combine variantions of these three OR conditions

# general filter
filter_pick = "pick"

# level 1 filter
filter_lof = "lof"

# level 2 filters
filter_polyphen2_hdiv_pred = "polyphen2_hdiv_pred"
filter_polyphen2_hvar_pred = "polyphen2_hvar_pred"
filter_sift_red = "sift_pred"
filter_mutationtaster_pred = "mutationtaster_pred"
filter_lrt_pred = "lrt_pred"
filter_metalr_pred = "metalr_pred"
filter_provean_pred = "provean_pred"
filter_fathmm_pred = "fathmm_pred"
filter_fathmm_mkl_coding_pred = "fathmm-mkl_coding_pred"
filter_eigen_pc_raw_rankscore = "eigen-pc-raw_rankscore"
filter_dann_rankscore = "dann_rankscore"
filter_vest3_rankscore = "vest3_rankscore"
filter_cadd_raw_rankscore = "cadd_raw_rankscore"
filter_metasvm_pred = "metasvm_pred"

# aliases w/o -
filter_fathmm_mkl_coding_pred_alias = "fathmm_mkl_coding_pred"
filter_eigen_pc_raw_rankscore_alias = "eigen_pc_raw_rankscore"

# level 3 filter
filter_impact = "impact"

# column constants
var_id = "varId"
gene_ensemble_id = "geneEnsembleId"
burden_bin_id = "burden_bin_id"


In [2]:
# variant list schema
all_schema = StructType(
    [
        StructField('gene_name', StringType(), nullable=False),
        StructField('gene_ensemble_id', StringType(), nullable=False),
        StructField('chromosome', StringType(), nullable=False),
        StructField('position', IntegerType(), nullable=False),
        StructField('burdenBinId', IntegerType(), nullable=False),
        StructField('varId', StringType(), nullable=False),
    ]
)

In [132]:
# open spark session
spark = SparkSession.builder.appName('bioindex').getOrCreate()


In [9]:
# load the json data
vep = spark.read.json(vep_srcdir)

# print
print("the loaded test data is:")
format(vep.show())


+-------------+-------------+---------+-----------------+--------------------+-----------------------+-----------------------+-------------------------------+---------------+---------+------+-----------------------+
|allele_string|assembly_name|      end|               id|               input|intergenic_consequences|most_severe_consequence|regulatory_feature_consequences|seq_region_name|    start|strand|transcript_consequences|
+-------------+-------------+---------+-----------------+--------------------+-----------------------+-----------------------+-------------------------------+---------------+---------+------+-----------------------+
|          A/G|       GRCh37|100011334| 10:100011334:A:G|10	100011334	1000...|                   null|       missense_variant|                           null|             10|100011334|     1|   [[Y/H, protein_co...|
|          G/A|       GRCh37|100015480| 10:100015480:G:A|10	100015480	1000...|                   null|       missense_variant|          

In [133]:
# create new data frame with only var id
transcript_consequences = vep.select(vep.id, vep.transcript_consequences) \
    .withColumn('cqs', explode(col('transcript_consequences'))) \
    .filter(col('cqs.pick') == 1) \
    .select(
        col('id').alias('varId'),
        col('cqs.gene_id').alias('geneEnsembleId'),
        col('cqs.' + filter_lof).alias(filter_lof),
        col('cqs.' + filter_pick).alias(filter_pick),
        col('cqs.' + filter_impact).alias(filter_impact),

        col('cqs.' + filter_polyphen2_hdiv_pred).alias(filter_polyphen2_hdiv_pred),
        col('cqs.' + filter_polyphen2_hvar_pred).alias(filter_polyphen2_hvar_pred),
        col('cqs.' + filter_sift_red).alias(filter_sift_red),
        col('cqs.' + filter_mutationtaster_pred).alias(filter_mutationtaster_pred),
        col('cqs.' + filter_lrt_pred).alias(filter_lrt_pred),
        col('cqs.' + filter_metalr_pred).alias(filter_metalr_pred),

        col('cqs.' + filter_provean_pred).alias(filter_provean_pred),
        col('cqs.' + filter_fathmm_pred).alias(filter_fathmm_pred),
        col('cqs.' + filter_fathmm_mkl_coding_pred).alias(filter_fathmm_mkl_coding_pred_alias),
        col('cqs.' + filter_eigen_pc_raw_rankscore).alias(filter_eigen_pc_raw_rankscore_alias),
        col('cqs.' + filter_dann_rankscore).alias(filter_dann_rankscore),
        col('cqs.' + filter_vest3_rankscore).alias(filter_vest3_rankscore),
        col('cqs.' + filter_cadd_raw_rankscore).alias(filter_cadd_raw_rankscore),
        col('cqs.' + filter_metasvm_pred).alias(filter_metasvm_pred),
    )


# print
print("the filtered test data is:")
transcript_consequences.show()



the filtered test data is:
+-------------------+---------------+----+----+--------+-------------------+-------------------+---------+-------------------+--------+-----------+------------+-----------+----------------------+----------------------+--------------+---------------+------------------+------------+
|              varId| geneEnsembleId| lof|pick|  impact|polyphen2_hdiv_pred|polyphen2_hvar_pred|sift_pred|mutationtaster_pred|lrt_pred|metalr_pred|provean_pred|fathmm_pred|fathmm_mkl_coding_pred|eigen_pc_raw_rankscore|dann_rankscore|vest3_rankscore|cadd_raw_rankscore|metasvm_pred|
+-------------------+---------------+----+----+--------+-------------------+-------------------+---------+-------------------+--------+-----------+------------+-----------+----------------------+----------------------+--------------+---------------+------------------+------------+
|   10:100011334:A:G|ENSG00000138131|null|   1|MODERATE|                  D|                  D|        D|                  D| 

In [113]:
transcript_consequences.lof

Column<b'lof'>

In [134]:
# get the lof level 1 data frame
dataframe_lof = transcript_consequences.filter(transcript_consequences.lof == 'HC')

# print
print("the lof data frame is")
dataframe_lof.show()

the lof data frame is
+--------------------+---------------+---+----+------+-------------------+-------------------+---------+-------------------+--------+-----------+------------+-----------+----------------------+----------------------+--------------+---------------+------------------+------------+
|               varId| geneEnsembleId|lof|pick|impact|polyphen2_hdiv_pred|polyphen2_hvar_pred|sift_pred|mutationtaster_pred|lrt_pred|metalr_pred|provean_pred|fathmm_pred|fathmm_mkl_coding_pred|eigen_pc_raw_rankscore|dann_rankscore|vest3_rankscore|cadd_raw_rankscore|metasvm_pred|
+--------------------+---------------+---+----+------+-------------------+-------------------+---------+-------------------+--------+-----------+------------+-----------+----------------------+----------------------+--------------+---------------+------------------+------------+
|    10:101460816:G:A|ENSG00000198018| HC|   1|  HIGH|               null|               null|     null|               null|    null|     

In [135]:
# get the level 3 dataframe
dataframe_impact_moderate = transcript_consequences.filter(transcript_consequences.impact == 'MODERATE')
dataframe_impact_high = transcript_consequences.filter(transcript_consequences.impact == 'HIGH')

# print
print("the moderate impact dataframe is {}".format(dataframe_impact_moderate.count()))
print("the high impact dataframe is {}".format(dataframe_impact_high.count()))
# dataframe_impact.show()


the moderate impact dataframe is 17407
the high impact dataframe is 1433


In [136]:
# get the initial level 2 dataframe
dataframe_level2 = transcript_consequences.filter(transcript_consequences.polyphen2_hdiv_pred != 'D') \
    .filter(transcript_consequences.polyphen2_hvar_pred != 'D') \
    .filter(transcript_consequences.sift_pred != 'deleterious') \
    .filter(transcript_consequences.lrt_pred != 'D') \
    .filter(~transcript_consequences.mutationtaster_pred.isin(['A', 'D']))

print("level 2 data frame count: {}".format(dataframe_level2.count()))
# dataframe_level2.show()


level 2 data frame count: 5611


In [137]:
# BIN 1 of 7
# create the final_1 df, just lof = HC
final_bin1_data_frame = dataframe_lof.withColumn("burden_bin_id", lit('bin1_7'))

# print
print("the final bin 1 dataframe is: {}".format(final_bin1_data_frame.count()))
# final_bin1_data_frame.show()

the final bin 1 dataframe is: 1201


In [138]:
# BIN 7 of 7
# get the initial level 2 dataframe
dataframe_level2 = transcript_consequences.filter(transcript_consequences.polyphen2_hdiv_pred != 'D') \
    .filter(transcript_consequences.polyphen2_hvar_pred != 'D') \
    .filter(transcript_consequences.sift_pred != 'deleterious') \
    .filter(transcript_consequences.lrt_pred != 'D') \
    .filter(~transcript_consequences.mutationtaster_pred.isin(['A', 'D']))

print("level 2 data frame count: {}".format(dataframe_level2.count()))
print("moderate impact data frame count: {}".format(dataframe_impact_moderate.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_7 df, lof = HC, impact moderate, add in level 2 filters
final_bin7_data_frame = dataframe_lof.union(dataframe_impact_moderate).union(dataframe_level2).distinct()
final_bin7_data_frame = final_bin7_data_frame.withColumn("burden_bin_id", lit('bin7_7'))

# print
print("the final bin 7 dataframe is: {}".format(final_bin7_data_frame.count()))
# final_bin7_data_frame.show()

level 2 data frame count: 5611
moderate impact data frame count: 17407
lof data frame count: 1201
the final bin 7 dataframe is: 18632


In [139]:
# BIN 6 of 7
# get the exclusion level 2 data frame
dataframe_level2_exclusion = transcript_consequences \
    .filter(transcript_consequences.polyphen2_hdiv_pred == 'D') \
    .filter(transcript_consequences.polyphen2_hvar_pred == 'D') \
    .filter(transcript_consequences.sift_pred == 'deleterious') \
    .filter(transcript_consequences.lrt_pred == 'D') \
    .filter(transcript_consequences.mutationtaster_pred.isin(['A', 'D']))

dataframe_level2_inclusion = transcript_consequences \
    .filter(transcript_consequences.polyphen2_hdiv_pred == 'D') \
    .union(transcript_consequences.filter(transcript_consequences.polyphen2_hvar_pred == 'D')) \
    .union(transcript_consequences.filter(transcript_consequences.sift_pred == 'deleterious')) \
    .union(transcript_consequences.filter(transcript_consequences.lrt_pred == 'D')) \
    .union(transcript_consequences.filter(transcript_consequences.mutationtaster_pred.isin(['A', 'D']))) \
    .distinct()

print("level 2 exclusion data frame count: {}".format(dataframe_level2_exclusion.count()))
print("level 2 inclusion data frame count: {}".format(dataframe_level2_inclusion.count()))
print("moderate impact data frame count: {}".format(dataframe_impact_moderate.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_6 df, lof = HC, impact moderate, add in level 2 filters
final_bin6_data_frame = dataframe_level2_inclusion.subtract(dataframe_level2_exclusion)
final_bin6_data_frame = final_bin6_data_frame.union(dataframe_lof).union(dataframe_impact_moderate).union(dataframe_level2_inclusion).distinct()
final_bin6_data_frame = final_bin6_data_frame.withColumn("burden_bin_id", lit('bin6_7'))

# print
print("the final bin 6 dataframe is: {}".format(final_bin6_data_frame.count()))
# final_bin7_data_frame.show()


level 2 exclusion data frame count: 0
level 2 inclusion data frame count: 9564
moderate impact data frame count: 17407
lof data frame count: 1201
the final bin 6 dataframe is: 18654


In [140]:
# BIN 5 of 7
# already have the inclusion level 2 data frame (exclusion from the previous bin 6 of 7)

print("level 2 inclusion data frame count: {}".format(dataframe_level2_exclusion.count()))
print("high impact data frame count: {}".format(dataframe_impact_high.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_5 df, lof = HC, impact moderate, add in level 2 filters
final_bin5_data_frame = dataframe_lof.union(dataframe_level2_exclusion).union(dataframe_impact_high).distinct()
final_bin5_data_frame = final_bin5_data_frame.withColumn("burden_bin_id", lit('bin5_7'))

# print
print("the final bin 5 dataframe is: {}".format(final_bin5_data_frame.count()))
# final_bin7_data_frame.show()

level 2 inclusion data frame count: 0
high impact data frame count: 1433
lof data frame count: 1201
the final bin 5 dataframe is: 1433


In [141]:
# BIN 4 of 7
# already have the inclusion level 2 data frame (exclusion from the previous bin 6 of 7)

print("level 2 inclusion data frame count: {}".format(dataframe_level2_exclusion.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_4 df, lof = HC, impact moderate, add in level 2 filters
final_bin4_data_frame = dataframe_lof.union(dataframe_level2_exclusion).distinct()
final_bin4_data_frame = final_bin4_data_frame.withColumn("burden_bin_id", lit('bin4_7'))

# print
print("the final bin 4 dataframe is: {}".format(final_bin4_data_frame.count()))
# final_bin7_data_frame.show()

level 2 inclusion data frame count: 0
lof data frame count: 1201
the final bin 4 dataframe is: 1201


In [142]:
# BIN 3 of 7
# bin consists of bin4 level 2 filter with some added on filters
dataframe_bin3_level2_inclusion = dataframe_level2_exclusion \
    .filter(dataframe_level2_exclusion.metalr_pred == 'D') \
    .filter(dataframe_level2_exclusion.metasvm_pred == 'D') \
    .filter(dataframe_level2_exclusion.provean_pred == 'D') \
    .filter(dataframe_level2_exclusion.fathmm_mkl_coding_pred == 'D') \
    .filter(dataframe_level2_exclusion.fathmm_pred == 'D')

print("bin 3 level 2 inclusion data frame count: {}".format(dataframe_bin3_level2_inclusion.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_3 df, lof = HC, add in level 2 filters
final_bin3_data_frame = dataframe_lof.union(dataframe_bin3_level2_inclusion).distinct()
final_bin3_data_frame = final_bin3_data_frame.withColumn("burden_bin_id", lit('bin3_7'))

# print
print("the final bin 3 dataframe is: {}".format(final_bin3_data_frame.count()))
# final_bin7_data_frame.show()

bin 3 level 2 inclusion data frame count: 0
lof data frame count: 1201
the final bin 3 dataframe is: 1201


In [143]:
# BIN 2 of 7
# bin consists of bin3 level 2 filter with some more added on filters
dataframe_bin2_level2_inclusion = dataframe_bin3_level2_inclusion \
    .filter(dataframe_level2_exclusion.eigen_pc_raw_rankscore > 0.9) \
    .filter(dataframe_level2_exclusion.dann_rankscore > 0.9) \
    .filter(dataframe_level2_exclusion.cadd_raw_rankscore > 0.9) \
    .filter(dataframe_level2_exclusion.vest3_rankscore > 0.9)

print("bin 2 level 2 inclusion data frame count: {}".format(dataframe_bin2_level2_inclusion.count()))
print("lof data frame count: {}".format(dataframe_lof.count()))
# dataframe_level2.show()

# create the final_2 df, lof = HC, add in level 2 filters
final_bin2_data_frame = dataframe_lof.union(dataframe_bin2_level2_inclusion).distinct()
final_bin2_data_frame = final_bin2_data_frame.withColumn(burden_bin_id, lit('bin2_7'))

# print
print("the final bin 3 dataframe is: {}".format(final_bin3_data_frame.count()))
# final_bin7_data_frame.show()

bin 2 level 2 inclusion data frame count: 0
lof data frame count: 1201
the final bin 3 dataframe is: 1201


In [144]:
# combine all the bins into one dataframe
output_data_frame = final_bin1_data_frame \
    .union(final_bin2_data_frame) \
    .union(final_bin3_data_frame) \
    .union(final_bin4_data_frame) \
    .union(final_bin5_data_frame) \
    .union(final_bin6_data_frame) \
    .union(final_bin7_data_frame).distinct().orderBy(var_id, gene_ensemble_id, burden_bin_id)
    # .distinct() \
    # .orderBy(var_id, gene_ensemble_id, burden_bin_id)

# print
print("the final agregated bin dataframe is: {}".format(output_data_frame.count()))


the final agregated bin dataframe is: 43523


In [145]:
# save out the output data frame to file
output_data_frame \
    .write \
    .mode('overwrite') \
    .json('%s/burden' % outdir)


In [ ]:
# filter_polyphen2_hdiv_pred = "polyphen2_hdiv_pred"
# filter_polyphen2_hvar_pred = "polyphen2_hvar_pred"
# filter_sift_red = "sift_pred"
# filter_mutationtaster_pred = "mutationtaster_pred"
# filter_lrt_pred = "lrt_pred"
# filter_metalr_pred = "metalr_pred"
# filter_provean_pred = "provean_pred"
# filter_fathmm_pred = "fathmm_pred"
# filter_fathmm_mkl_coding_pred = "fathmm_mkl_coding_pred"
# filter_eigen_pc_raw_rankscore = "eigen_pc_raw_rankscore"
# filter_dann_rankscore = "dann_rankscore"
# filter_vest3_rankscore = "vest3_rankscore"
# filter_cadd_raw_rankscore = "cadd_raw_rankscore"
# filter_metasvm_pred = "metasvm_pred"

# aliases w/o -
# filter_fathmm_mkl_coding_pred_alias = "fathmm_mkl_coding_pred"
# filter_eigen_pc_raw_rankscore_alias = "eigen_pc-raw_rankscore"
